In [ ]:
# cloud function to update excel file
import openpyxl
import json
import requests

# Cloud function handler
def update_inventory(event, context):
    data = json.loads(event['body'])  # Extract scanned data
    product_id = data.get('product_id')
    change_in_quantity = data.get('quantity_change')

    # Load the Excel file
    workbook = openpyxl.load_workbook('/path/to/inventory.xlsx')
    sheet = workbook.active

    # Find the product row by product_id and update inventory
    for row in sheet.iter_rows(min_row=2, values_only=True):
        if row[0] == product_id:
            current_qty = row[2]
            new_qty = current_qty + change_in_quantity
            sheet[f'C{row[0]}'] = new_qty  # Assuming column C is the stock level
            break

    # Save updated Excel file
    workbook.save('/path/to/inventory.xlsx')

    return {'statusCode': 200, 'body': json.dumps('Inventory updated')}


In [ ]:
# create function to create qrcode that holds url, product_id and quantity_change
def create_qrcode(event, context):
    data = json.loads(event['body'])  # Extract scanned data
    product_id = data.get('product_id')
    quantity_change = data.get('quantity_change')
    url = f"https://example.com/update_inventory?product_id={product_id}&quantity_change={quantity_change}"
    qr_code = requests.get(f"https://api.qrserver.com/v1/create-qr-code/?data={url}&size=200x200")
    return {'statusCode': 200, 'body': json.dumps(qr_code.content)}

# create qr code in python
def create_qr_code(url):
    qr_code = requests.get(f"https://api.qrserver.com/v1/create-qr-code/?data={url}&size=200x200")
    return qr_code.content